## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Determine the number of unique values in each column.

print(application_df.apply(lambda col: len(col.unique())))

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_types = application_df["APPLICATION_TYPE"].value_counts()
print(app_types)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = app_types[app_types<500].axes
print(application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

[Index(['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object', name='APPLICATION_TYPE')]


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts[classification_counts< 1000].axes

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [ ]:
# Look at INCOME_AMT value counts for binning
income_types = application_df["INCOME_AMT"].value_counts()
print(income_types)

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64


In [ ]:
# Bin Income_Amt in range 1-9999 and 10000-24999 as 0
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace("1-9999","0")

application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace("10000-24999","0")

In [ ]:
# Bin Income_Amt in range 1m and above as 1M+
income_types_to_replace = income_types[income_types.index.str.contains('M')].axes
income_types_to_replace

# Replace in dataframe
for inc in income_types_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(inc,"1M+")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                25659
25000-99999       3747
100000-499999     3374
1M+               1519
Name: count, dtype: int64

In [ ]:
# Look at AFFILIATION value counts for binning
affiliation_types = application_df["AFFILIATION"].value_counts()
print(affiliation_types)

AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of affiliations to be replaced

affiliation_types_to_replace = affiliation_types[affiliation_types<1000].axes
print(affiliation_types_to_replace)

# Replace in dataframe
for afl in affiliation_types_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(afl,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

[Index(['Family/Parent', 'National', 'Regional', 'Other'], dtype='object', name='AFFILIATION')]


AFFILIATION
Independent         18480
CompanySponsored    15705
Other                 114
Name: count, dtype: int64

In [ ]:
# Look at USE_CASE value counts for binning
use_case_types = application_df["USE_CASE"].value_counts()
print(use_case_types)

USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of use cases to be replaced

use_case_types_to_replace = use_case_types[use_case_types<1000].axes
print(use_case_types_to_replace)

# Replace in dataframe
for usecase in use_case_types_to_replace:
   application_df['USE_CASE'] = application_df['USE_CASE'].replace(usecase,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

[Index(['CommunityServ', 'Heathcare', 'Other'], dtype='object', name='USE_CASE')]


USE_CASE
Preservation    28095
ProductDev       5671
Other             533
Name: count, dtype: int64

In [ ]:
# Look at ORGANIZATION value counts for binning
org_types = application_df["ORGANIZATION"].value_counts()
print(org_types)

ORGANIZATION
Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: count, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of affilications to be replaced
org_types_to_replace = org_types[org_types<1000].axes
print(org_types_to_replace)

# Replace in dataframe
for org in org_types_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(org,"Other")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

[Index(['Co-operative', 'Corporation'], dtype='object', name='ORGANIZATION')]


ORGANIZATION
Trust          23515
Association    10255
Other            529
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dtype=int)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_1M+,INCOME_AMT_25000-99999,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,1,1,0,0,1,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,1,1,0,0,1,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,1,0


In [ ]:
print(application_df.columns)

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Independent',
       'AFFILIATION_Other', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_Other', 'USE_CASE_Preservation',
       'USE_CASE_ProductDev', 'ORGANIZATION_Association', 'ORGANIZATION_Other',
       'ORGANIZATION_Trust', 'INCOME_AMT_0', 'INCOME_AMT_100000-499999',
       'INCOME_AMT_1M+', 'INCOME_AMT_25000-99999', 'SPECIAL_CONSIDERATIONS_N',
       'SPECIAL_CONSIDERATIONS_Y'],
      dtype='object')


In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)



In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Create a list to save the weights after every 5 epochs
weights_saved = []

In [ ]:
# Create a custom callback to save the weights every five epochs
class CustomCallback(tf.keras.callbacks.Callback):

   # Save the weights every five epochs
   def on_epoch_end(self, epoch, logs=None):

        if epoch % 5 == 0:
          weights_saved.append(self.model.get_weights())
          print(f"\n Epoch {epoch} ended - saving weights..")

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="relu", input_dim=len(X_train.columns)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=24, activation="relu", input_dim=len(X_train.columns)))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 60)                1980      
                                                                 
 dense_4 (Dense)             (None, 24)                1464      
                                                                 
 dense_5 (Dense)             (None, 1)                 25        
                                                                 
Total params: 3469 (13.55 KB)
Trainable params: 3469 (13.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
nn.fit(X_train, y_train, epochs=100, callbacks=[CustomCallback()])

Epoch 1/100
796/804 [============================>.] - ETA: 0s - loss: 30607.0117 - accuracy: 0.4954
 Epoch 0 ended - saving weights..
804/804 [==============================] - 3s 3ms/step - loss: 32038.5293 - accuracy: 0.4957
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 63068.4141 - accuracy: 0.4804
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 26494.7402 - accuracy: 0.4895
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 11993.1455 - accuracy: 0.4970
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 35996.0625 - accuracy: 0.5058
Epoch 6/100
787/804 [============================>.] - ETA: 0s - loss: 27300.7500 - accuracy: 0.5072
 Epoch 5 ended - saving weights..
804/804 [==============================] - 2s 2ms/step - loss: 27971.9531 - accuracy: 0.5069
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 17819.9766 - accuracy: 0.5071
Epoch 8/100
804/804

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.8298 - accuracy: 0.6780 - 580ms/epoch - 2ms/step
Loss: 0.8297837972640991, Accuracy: 0.6780174970626831


In [ ]:
# Export our model to HDF5 file
nn.save("alphabet_soup_charity_optimization_model.h5", save_format="h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
